In [1]:
# Algorithm:
# read the position and velocity data
# sort position data by Z-position

# For Longitudinal acoustic mode, use LA mode frequency from dispersion relation
# Calculate and add displacement in z-direction and velocity in the z-direction as well

# For Transverse acoustic mode along x-dirn, use TAx mode frequency from dispersion relation
# Calculate and add displacement in x-direction and velocity in the x-direction as well

# Remember the structure file produced will have atom id in floating point, put it into ovito
# save as atomic full style, then it will correct it. Just a final work around. But it will
# make all velocity values zero, so copy it from the original file created from here.

In [2]:
import numpy as np
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
import cmath

In [3]:
# Phonon dispersion relation values:
# These values should be obtained from Phonon-dispersion relation of Au 

# Calculated by XW:
# TA_freq = np.array([0, 422531609399.84, 834200909849.15, 1224563660601.91, 1583947824780.52, 
#                  1903668508126.36, 2176185219999.14, 2395114621687.84, 2555327626841.41, 
#                  2653016453446.38, 2685847412572.21])  # frequency in Hz
# LA_freq = np.array([0, 928780803795.77, 1849542776769.78, 2750148874370.31, 3611995172904.95,
#                         4410212916740.89, 5116139573866.81, 5700820244640.96, 6138553474768.24,
#                         6409474658336.53, 6501180691475.74])

# Along 000 ==> 111 direction
LA_freq = np.array([0.00,925461626247.93,1843126160310.45,2741039078132.37,3600698718147.45,
                    4397269403736.85,5102035424373.30,5685976180599.75,6123273156203.04,
                    6393976302314.61,6485618869485.52])

TA_freq = np.array([0.00,419656228038.71,828716471524.88,1216957807640.71,1574837572634.86,
                    1893702257207.15,2165911021001.29,2384922632417.69,2545398447322.77,
                    2643344720528.80,2676275393098.56])

k_val = np.arange(0.0, 1.1, 0.1); print(k_val[10])

# TA_freq_THz = TA_freq/1e12; 
# LA_freq_THz = LA_freq/1e12;

# plt.plot(k_val, TA_freq_THz, 'o-r', label='TA mode')
# plt.plot(k_val, LA_freq_THz, 'o-b', label='LA mode')
# plt.xlabel('k values [$\pi$/a]')
# plt.ylabel('Frequency [THz]')
# plt.title('Phonon dispersion relation of Au, (000) to (111) ')
# plt.legend()
# plt.show()

1.0


In [4]:
# Input variables:
a0 = 4.079016               # lattice parameter of Au, a = 4.0782(lit), 4.07901835(calc)
A = 0.002                   # Amplitude of the wave in angstrom, 2e-3
eps_k0 = 1                  # e(k0) = Polarization wave vector, why 1?

val = 10

# a = a0                           # Case-1     
# a = a0*np.sqrt(3)              # Case-2
a = a0/np.sqrt(3)              # Case-3

# a = a0                                # Case-4
# w_LA = 2*np.pi*LA_freq[val]           # in Hz, LA freq corresponding to k0 = 0.9*pi/a

gamma = 50*a0               # Spatial extent

k0 = k_val[val]*np.pi/a               # wave vector, k0 = 0.9*pi/a

w_LA = 2*np.pi*LA_freq[val]           # in Hz, LA freq corresponding to k0 = 0.9*pi/a
w_TA = 2*np.pi*TA_freq[val]           # in Hz, TA freq corresponding to k0 = 0.9*pi/a

# Structural variable:
N_atoms = 358560  # Number of atoms
N_layer = 996     # Number of Au(111) layers
N_layeratom = 360 # Number of atoms in each Au(111) layer N_atoms/N_layer

print("LA freq:", LA_freq[val]/1e12,"THz", "TA freq: ", TA_freq[val]/1e12, "THz")

LA freq: 6.48561886948552 THz TA freq:  2.67627539309856 THz


In [5]:
offset_p = 19
N_atoms = 358560

# print("Original position: data.npt50psaurelax_EM")
with open("data_cg_boxrelax.npt50psaurelax_EM", "r") as file:   # jid=253256
    lines = file.readlines()

# Read position data:
plines = lines[offset_p:(N_atoms+offset_p)]
# Extract data from line 19 to 358579
position = np.array([list(map(float, line.strip().split())) for line in plines])
print("Shape of position:", position.shape)

# Read velocity data:
offset_v = N_atoms+offset_p + 3
vlines = lines[offset_v:(N_atoms+offset_v)]
# Extract data from line 19 to 358579
velocity = np.array([list(map(float, line.strip().split())) for line in vlines])
print("Shape of velocity:", velocity.shape)

Shape of position: (358560, 10)
Shape of velocity: (358560, 4)


In [6]:
# Sorting the position according to Z-position:
sorted_index = np.argsort(position[:, 6])
position = position[sorted_index]            # col 4 = x, col 5 = y, col 6 = z

# Sorting the velocity according to atomid:
v_index = np.argsort(velocity[:, 0])    # sorting according to atom id
velocity = velocity[v_index];
vx = velocity[:, 1]
vy = velocity[:, 2]
vz = velocity[:, 3]

In [7]:
maxZ = np.max(position[:, 6]); print('Maximum Z-position:', maxZ)
minZ = np.min(position[:, 6]); print('Minimum Z-position:', minZ)

# position[:, 6] = position[:, 6] - minZ       # shifting the lower limit to zero

# plt.figure(figsize =(25, 5))
# plt.plot(position[:, 6], position[:, 5], '.', markersize=0.1)
# plt.xlabel('Z-position [A]')
# plt.ylabel('Y-position [A]')
# plt.show()

Maximum Z-position: 2340.6464846223594
Minimum Z-position: -2.6007416136501784


In [8]:
N_layers = position.shape[0] // N_layeratom   # Number of Au layers

# initialize the new array
Au_layers = np.zeros((N_layers, 1)) # Creating empty array

# loop over the number of averages
for i in range(N_layers):
    start = i * N_layeratom
    end = (i + 1) * N_layeratom
    
    # calculate the average of every 360 rows
    avg = np.mean(position[start:end, 6], axis=0)
    
    # store the average in the new array
    Au_layers[i, 0] = avg
Au_layers = Au_layers.reshape(-1)         

In [9]:
# Computing the displacements
xn = Au_layers                             # All positions
x0 = xn[xn.shape[0]//2]      # middle of the Au layer, for 2600 A

In [10]:
y = 0 + k0*(xn - x0)*1j
disp = A*eps_k0*np.exp(-(xn - x0)**2/gamma**2)*np.exp(y)

# Plot the position
U = np.real(disp)                        # Take the real part for displacement
Uimg = np.imag(disp)
plt.plot(xn, U, label='real')
# plt.plot(xn, Uimg, label='imag')
plt.xlabel("Z-axis coordinate [A]")
plt.ylabel("Displacement [A]")
# plt.xlim(800, 900)
# plt.ylim(-0.2, 0.2)
plt.axhline(0.002, color='k', ls='--')
plt.axhline(-0.002, color='k', ls='--')
plt.savefig("Wave-packet_displacement.png", dpi= 200, bbox_inches = 'tight', facecolor="w")
plt.show()


# Plot the velocity
vel = (w_LA*np.imag(disp))/1e12           # For LA mode set this as velocity in z-dirn, A/ps            
plt.plot(xn, vel)
plt.xlabel("Z-axis coordinate [A]")
plt.ylabel("velocity, v [A/ps]")
plt.savefig("Wave-packet_velocity.png", dpi= 200, bbox_inches = 'tight', facecolor="w")
# plt.xlim(500, 550)
# plt.ylim(-0.5, 0.5)
plt.show()

# np.savetxt("Wave-packet_displacement.dat", U)
# np.savetxt("Wave-packet_velocity.dat", vel) 

In [11]:
# Repeating each of these values 360 times for all the atoms in each Au layer
new_pos = (U.reshape(U.shape[0], 1) @ np.ones(N_layeratom).reshape(1, N_layeratom)).flatten()
new_vel = (vel.reshape(vel.shape[0], 1) @ np.ones(N_layeratom).reshape(1, N_layeratom)).flatten()

In [12]:
# Now let's do the replacement of both position and velocity:

# position
# Adding displacement to the Z-position 
# new_z = position[:, 6] + new_pos
position[:, 6] = 1.0000000000*new_pos + 1.0000000000*position[:, 6]

phonon_pos = position[:, 0:7]                     # only taking from 0 to 6th column
p_index = np.argsort(phonon_pos[:, 0])
phonon_pos = phonon_pos[p_index]

# velocity
# replacing vz component of velocity, vx = vy = 0 A/ps for LA mode
velocity[:, 0] = position[:, 0]                 # atom ids
# velocity[:, 1] = vx                             # vx = 0 A/ps
# velocity[:, 2] = vy                             # vy = 0 A/ps
# velocity[:, 3] = vz + 1.0000000000*new_vel      # vz = computed velocity
velocity[:, 1] = np.zeros(velocity.shape[0])    # vx = 0 A/ps
velocity[:, 2] = np.zeros(velocity.shape[0])    # vy = 0 A/ps
velocity[:, 3] = 1.0000000000*new_vel           # vz = computed velocity

# Sorting the velocity according to atomid:
v_index = np.argsort(velocity[:, 0])
velocity = velocity[v_index];
# velocity = np.abs(velocity[v_index]); 

In [13]:
# os.system('rm Au_111_18x10x332*.lmp')

with open("Au_111_18x10x332_A002_k10_abysqrt3.lmp", "w") as output_file:
    
    output_file.write(lines[0])
    output_file.write(lines[2])
    output_file.write(lines[3])
    
    for i in range(5, 13):
        output_file.write(lines[i])

    for i in range(17, 19):
        output_file.write(lines[i])

    for item in phonon_pos:
        output_file.write(str(int(item[0]))+ '\t' + str(int(item[1]))+ '\t' +
                          str(int(item[2]))+ '\t' + 
                          '\t'.join(str(elem) for elem in item[3:]) + '\n')
#         output_file.write(item_arr + "\n")
    
    output_file.write("\n")
    output_file.write("Velocities"+ "\n")
    output_file.write("\n")
    
    for item in velocity:
        output_file.write(str(int(item[0]))+ '\t' + '\t'.join(str(elem) for elem in item[1:]) + '\n')
#         item_arr = " ".join(str(elem) for elem in item)
#         output_file.write(item_arr + "\n")

       
# np.savetxt("Velocity.dat", velocity)
# np.savetxt("Displacement.dat", phonon_pos)


In [14]:
print('Input paramters:')
print('k-value:', k_val[val])
print('Selected frequency:', LA_freq[val]/1e12, 'THz')
print('Lattice parameter:', a, 'Ang')
print('Center of wave packet:', x0)

Input paramters:
k-value: 1.0
Selected frequency: 6.48561886948552 THz
Lattice parameter: 2.3550209856287907 Ang
Center of wave packet: 1170.2003833792658


In [15]:
# Sorting the position according to Z-position:
sorted_index = np.argsort(position[:, 0])
position = position[sorted_index]            # col 4 = x, col 5 = y, col 6 = z
np.savetxt("z_disp_vel.dat", np.hstack((position[:, 6][:, None], 
                                        phonon_pos[:, 6][:, None], velocity[:, 3][:, None])))

# Creating directory and moving file:
os.system('mkdir A002_k10_abysqrt3')
os.system('mv *.dat *.png *.lmp A002_k10_abysqrt3')

0